In [ ]:
import pandas as pd

In [ ]:
gema_extended_filename_path = "../data/events/gema/Veranstaltungen Freiburg Dresden_10.03.2025.xlsx"
gema_1 = pd.read_excel(gema_extended_filename_path, sheet_name="06-07 2023")
gema_2 = pd.read_excel(gema_extended_filename_path, sheet_name="01-03 2024")
gema_3 = pd.read_excel(gema_extended_filename_path, sheet_name="09-10 2024")

In [ ]:
gema_1

In [ ]:
df_gema_extended_events = pd.concat([gema_1, gema_2, gema_3], ignore_index=True)

In [ ]:
len(df_gema_extended_events)

In [ ]:
df_gema_extended_events.info()

In [ ]:
df_gema_extended_events.Latitude = df_gema_extended_events.Latitude.str.replace(",",".").astype(float)
df_gema_extended_events.Longitude = df_gema_extended_events.Longitude.str.replace(",",".").astype(float)

In [ ]:
df_gema_extended_events.head(10)

In [ ]:
df_gema_extended_events.columns = [col.replace(" ", "_") for col in df_gema_extended_events.columns]
    

In [ ]:
df = pd.read_pickle("../data/events/df_events with full StartDateTime 2025-03-02_18-01-52.pkl")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(columns=["StartDate", "StartTime", "StartMonth" ], errors="ignore", inplace=True)

In [ ]:
df.loc[df.Source=="GEMA", "origin_id"]

In [ ]:
df.info()

In [ ]:
flt = df.Source=="GEMA" 
df.loc[flt, "origin_id"] = df.loc[flt, "origin_id"].astype(int) 

In [ ]:
gema_veranst_id = df_gema_extended_events["Veranst_ID"].tolist()

In [ ]:
flt = (df.Source=="GEMA") & (df.origin_id.isin(gema_veranst_id))
len(df[flt])

In [ ]:
df_gema_extended_events.columns = [col+"_GEMA" for col in df_gema_extended_events.columns]

In [ ]:
df_gema_extended_events.rename(columns={"Veranst_ID_GEMA":"origin_id"}, inplace=True, errors="ignore")

In [ ]:
df_gema_extended_events

In [ ]:
len(df)

In [ ]:
df.merge(df_gema_extended_events, how="left", on="origin_id", suffixes=("", "_GEMA"), indicator=True).query("_merge == 'both'").Source.unique()

In [ ]:
df = df.merge(df_gema_extended_events, how="left", on="origin_id", suffixes=("", "_GEMA"), indicator=True)

In [ ]:
df.columns

In [ ]:
df.query("_merge == 'both'")[["EndDateTime", "Datum_Nutzungsende_GEMA"]]

In [ ]:
flt = (df._merge == 'both') & (~df.Datum_Nutzungsbeginn_GEMA.isna())
df.loc[flt, "StartDateTime"] = df.loc[flt, "Datum_Nutzungsbeginn_GEMA"]

In [ ]:
flt = (df._merge == 'both') & (~df.Latitude_GEMA.isna())
df.loc[flt, "lat"] = df.loc[flt, "Latitude_GEMA"]
df.loc[flt, "lon"] = df.loc[flt, "Longitude_GEMA"]

In [ ]:
df.columns

In [ ]:
flt = (df._merge == 'both') & (~df.Datum_Nutzungsende_GEMA.isna())

df.loc[flt, "EndDateTime"] = df.loc[flt, "Datum_Nutzungsende_GEMA"]

In [ ]:
flt = df_gema_extended_events.origin_id.isin(df.origin_id.tolist())
df_events_not_present = df_gema_extended_events[~flt]

In [ ]:
df_events_not_present.columns

In [ ]:
flt =( df_events_not_present.Datum_Nutzungsbeginn_GEMA.dt.year==2023) & (df_events_not_present.Ort_GEMA=="Dresden")
df_events_not_present = df_events_not_present[~flt]


In [ ]:
df.lat.isna().value_counts()

In [ ]:
flt = df.lat.isna()
df = df[~flt]

In [ ]:
columns_to_keep = ["origin_id", "StartDateTime", "EndDateTime", "lat", "lon", "Source", "SourceGroup"]

In [ ]:
flt = (df.EndDateTime.isna()) & ~(df.EndTime.isna()) 
df[  flt]

In [ ]:
from datetime import timedelta
from datetime import datetime

In [ ]:
def get_end_date_time(row):
    end_time_str = row["EndTime"]
    if row["EndTime"] == "24:00":
        end_time_str = "0:00"
    
    end_time = pd.to_datetime(end_time_str, format="%H:%M").time()

    date_part = row["StartDateTime"]
    if end_time_str == "0:00":
        date_part = date_part + timedelta(days=1)

    end_date_time = datetime.combine(date_part, end_time)
    return end_date_time
    


    


In [ ]:
flt = df.EndDateTime.isna() & df.EndTime.notna()
values = df[flt].apply(get_end_date_time, axis=1)
df.loc[flt, "EndDateTime"] = values

In [ ]:
flt = df.EndDateTime == df.StartDateTime
df.loc[flt, "EndDateTime"] = pd.NA

In [ ]:
df = df[columns_to_keep]

In [ ]:
df

In [ ]:
import h3

In [ ]:
resolution = 7

In [ ]:
df["hex_id"] = df.apply(lambda row: h3.latlng_to_cell(row["lat"], row["lon"], resolution), axis=1)

In [ ]:
df

In [ ]:
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden 2025-03-19_10-47-56.csv"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg_missing_interpolated_2025-03-19_10-47-56.csv"
df_DD_bikes = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_FB_bikes = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])

In [ ]:
existing_hex_ids = df_DD_bikes.hex_id.unique().tolist()

In [ ]:
existing_hex_ids = existing_hex_ids + df_FB_bikes.hex_id.unique().tolist()

In [ ]:
len(existing_hex_ids)

In [ ]:
flt = df.hex_id.isin(existing_hex_ids)
df = df[flt]

In [ ]:
len(df)

In [ ]:
df

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
df.Source.unique()

In [ ]:
time

In [ ]:

df.to_csv(f"../data/events/df_events_with_hex_id_{time}.csv", index=False)